# 양방향 RNN을 이용한 품사 태깅

PyTorch 시퀸스 레이블링의 대표적 테스크 품사 태깅 (PoS tagging)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.legacy import data, datasets
import time
import random

In [8]:
# 랜덤 시드 고정
# 랜덤 시드 1234보다 3407이 정확도 면에서 훨씬 좋다
SEED = 3407
random.seed(SEED)
torch.manual_seed(SEED)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. 필드 정의하기

총 3개의 필드 정의

In [18]:
TEXT = data.Field(lower = True)
UD_TAGS = data.Field(unk_token = None)
PTB_TAGS = data.Field(unk_token = None)

fields = (("text", TEXT), ("udtags", UD_TAGS), ("ptbtags", PTB_TAGS))

# 2. 데이터셋 만들기

1. 훈련 데이터

2. 검증 데이터

3. 테스트 데이터

In [19]:
train_data, valid_data, test_data = datasets.UDPOS.splits(fields)

In [20]:
print(f"훈련 샘플의 개수 : {len(train_data)}")
print(f"검증 샘플의 개수 : {len(valid_data)}")
print(f"테스트 샘플의 개수 : {len(test_data)}")

훈련 샘플의 개수 : 12543
검증 샘플의 개수 : 2002
테스트 샘플의 개수 : 2077


In [21]:
# 훈련 데이터의 3개의 필드 확인
print(train_data.fields)

{'text': <torchtext.legacy.data.field.Field object at 0x000001A988A0BAF0>, 'udtags': <torchtext.legacy.data.field.Field object at 0x000001A988A0B970>, 'ptbtags': <torchtext.legacy.data.field.Field object at 0x000001A988A0B0A0>}


In [22]:
# 첫번째 훈련 샘플의 text 필드
print(vars(train_data.examples[0])['text'])

['al', '-', 'zaman', ':', 'american', 'forces', 'killed', 'shaikh', 'abdullah', 'al', '-', 'ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'qaim', ',', 'near', 'the', 'syrian', 'border', '.']


In [23]:
# 첫번째 훈련 샘플의 udtags 필드
print(vars(train_data.examples[0])['udtags'])

['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT']


In [24]:
# 첫번째 훈련 샘플의 ptbdtags 필드
print(vars(train_data.examples[0])['ptbtags'])

['NNP', 'HYPH', 'NNP', ':', 'JJ', 'NNS', 'VBD', 'NNP', 'NNP', 'NNP', 'HYPH', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'IN', 'DT', 'JJ', 'NN', '.']


# 3. 단어 집합 (vocabulary) 만들기

단어 집합 생성시 사전 훈련된 워드 임베딩 GloVe 사용

In [25]:
# 최소 허용 빈도
MIN_FREQ = 5

# 사전 훈련된 워드 임베딩 GloVe 다운로드
TEXT.build_vocab(train_data, min_freq = MIN_FREQ, vectors = "glove.6B.100d")
UD_TAGS.build_vocab(train_data)
PTB_TAGS.build_vocab(train_data)

.vector_cache\glove.6B.zip: 862MB [09:09, 1.57MB/s]                               
100%|█████████▉| 399999/400000 [00:57<00:00, 6924.27it/s]


In [26]:
# 상위 빈도수 20개 단어
print(TEXT.vocab.freqs.most_common(20))

[('the', 9076), ('.', 8640), (',', 7021), ('to', 5137), ('and', 5002), ('a', 3782), ('of', 3622), ('i', 3379), ('in', 3112), ('is', 2239), ('you', 2156), ('that', 2036), ('it', 1850), ('for', 1842), ('-', 1426), ('have', 1359), ('"', 1296), ('on', 1273), ('was', 1244), ('with', 1216)]
